<a href="https://colab.research.google.com/github/Archangel212/GAN_jupyter_notebooks/blob/master/DCGAN_Batik_Dataset64by64_500LatentDim_pflip_changed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

sys.path.append("/content/drive/My Drive")
import numpy as np
import tensorflow as tf 

np.random.seed(1)
tf.random.set_seed(2)
# dcgan on batik dataset
# %tensorflow_version 1.0
# from numpy.random import randn
# from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential,load_model,save_model
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.utils import plot_model
import keras.backend as K
from matplotlib import pyplot as plt
from utils.save_model_summary import save_model_summary
from utils.get_last_epoch import get_last_epoch
from utils.trim_csv import trim_csv
from PIL import ImageFont
import io
import h5py
import os
import time
import pandas as pd
import threading
import re
import keras



print("Keras version: %s" % (keras.__version__))
print("Tensorflow version: %s" % (tf.__version__))
os.chdir("/content/drive/My Drive/DCGAN_Batik_Dataset/DCGAN_64by64_best_practice_500LatentDimension_pflip_changed")
# os.chdir("/content/drive/My Drive/DCGAN_Batik_Dataset/test")
os.getcwd()

Keras version: 2.3.1
Tensorflow version: 2.2.0


'/content/drive/My Drive/DCGAN_Batik_Dataset/test'

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(64,64,3)):
  init = RandomNormal(mean=0.0,stddev=0.02)
  model = Sequential()
  # normal
  model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape,kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # downsample 32x32
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # downsample 16x16
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # downsample 8x8
  model.add(Conv2D(128, (3,3), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # downsample 4x4
  model.add(Conv2D(256, (3,3), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model



In [4]:
# define the standalone generator model
def define_generator(latent_dim):
  init = RandomNormal(mean=0.0, stddev=0.02)
  model = Sequential()
  # foundation for 4x4 image
  n_nodes = 256 * 4 * 4
  model.add(Dense(n_nodes, input_dim=latent_dim, kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  model.add(Reshape((4, 4, 256)))
  # upsample to 8x8
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # upsample to 16x16
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # upsample to 32x32
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # upsample to 64x64
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(axis=-1))
  # output layer
  model.add(Conv2D(3, (3,3), activation='tanh', padding='same',kernel_initializer=init))
  return model


In [5]:

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  # connect them
  model = Sequential()
  # add generator
  model.add(g_model)
  # add the discriminator
  model.add(d_model)
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model


In [6]:
def load_dataset(ds_path):
  with h5py.File(ds_path,"r") as f:
    dataset = f["Batik"]
    dataset = np.copy(dataset)
  return dataset

# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
  ds_path = "/content/drive/My Drive/Batik_Datasets/pinterest_version/batik_dataset_64by64.hdf5"
  trainX = load_dataset(ds_path)
  # # (trainX, _), (_, _) = load_data()
  # convert from unsigned ints to floats
  X = trainX.astype('float32')
  # scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  return X

def noisy_labels(y, p_flip):
  # ix = np.random.choice(y.shape[0], size=int(y.shape[0]*p_flip), replace=False)
  # y[ix] = 1 - y[ix]
  n_select = int(p_flip * y.shape[0])
  # choose labels to flip
  flip_ix = np.random.choice([i for i in range(y.shape[0])], size=n_select, replace=False)
  # invert the labels in place
  y[flip_ix] = 1 - y[flip_ix]
  return y


# select real samples
def generate_real_samples(dataset, n_samples, label_smoothing=True, label_noising=True, p_flip=0.01):
  # choose random instances
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  # retrieve selected images

  X = dataset[ix]
  # generate 'real' class labels (1)
  y = np.ones((n_samples, 1))
  if label_smoothing:
    y = y - 0.3 + (np.random.random(y.shape) * 0.5) 
  if label_noising:
    y = noisy_labels(y, p_flip)
  return X, y


In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples,label_noising=True,p_flip=0.01):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = np.zeros((n_samples, 1))
	if label_noising:
		y = noisy_labels(y,p_flip)
	return X, y


In [8]:

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, gan_model, dataset, latent_dim, n_samples=150):
  # prepare real samples
  X_real, y_real = generate_real_samples(dataset, n_samples,label_smoothing=False)
  # evaluate discriminator on real examples
  _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
  # prepare fake examples
  x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  # evaluate discriminator on fake examples
  _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
  # summarize discriminator performance
  print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
  # save plot
  save_plot(x_fake, epoch)
  # save the generator model tile file
  gen_filename = 'generator_model_%03d.h5' % (epoch+1)
  gen_filename = os.path.join("model_checkpoints",gen_filename)

  dis_filename = 'discriminator_model.h5'
  dis_filename = os.path.join("model_checkpoints",dis_filename)
  
  gan_filename = 'GAN_model_weights.h5'
  gan_filename = os.path.join("model_checkpoints",gan_filename)

  gan_opt_weights_fname = 'GAN_optimizer_weights.npy'
  gan_opt_weights_fname = os.path.join("model_checkpoints",gan_opt_weights_fname)


  symbolic_weights = gan_model.optimizer.get_weights()
  np.save(gan_opt_weights_fname, symbolic_weights)

  gan_model.save_weights(gan_filename) 
  save_model(g_model,gen_filename)
  save_model(d_model,dis_filename)


In [9]:

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
  # scale from [-1,1] to [0,1]
  examples = (examples + 1) / 2.0
  # plot images
  for i in range(n * n):
    # define subplot
    plt.subplot(n, n, 1 + i)
    # turn off axis
    plt.axis('off')
    # plot raw pixel data
    plt.imshow(examples[i])
  # save plot to file
  filename = 'generated_plot_e%03d.png' % (epoch+1)
  filename = os.path.join("figure_plots",filename)
  plt.savefig(filename)
  plt.close()

def plot_history(df_hist):
  plt.subplot(2,1,1)
  plt.plot(df_hist["d_loss_real"],label="d_loss_real")
  plt.plot(df_hist["d_loss_fake"],label="d_loss_fake")
  plt.plot(df_hist["g_loss"],label="g_loss")
  plt.xlabel("number of iterations")
  plt.ylabel("loss")
  plt.legend()

  plt.subplot(2,1,2)
  plt.plot(df_hist["d_acc_real"], label="d_acc_real")
  plt.plot(df_hist["d_acc_fake"], label="d_acc_fake")
  plt.xlabel("number of iterations")
  plt.ylabel("accuracy")
  plt.legend()
  
  plt.savefig("accuracy_loss_plot.jpg")
  plt.close()

In [10]:

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim,csv_path=None, n_epochs=200, n_batch=128, initial_epoch=0):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  df_hist = pd.read_csv(csv_path,index_col=None)
  list_hist = df_hist.values.tolist()

  # manually enumerate epochs
  for i in range(initial_epoch,n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected 'real' samples
      X_real, y_real = generate_real_samples(dataset, half_batch,label_smoothing=True)
      # update discriminator model weights
      d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)

      # generate 'fake' examples
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator model weights
      d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)

      # prepare points in latent space as input for the generator
      X_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = np.ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch(X_gan, y_gan)

      list_hist.append([i+1, 
                        "%d/%d" % (j+1,bat_per_epo), 
                        d_acc1, 
                        d_loss1, 
                        d_acc2, 
                        d_loss2,
                        g_loss])
      

      # summarize loss on this batch
      print('>%d, %d/%d, d1=%.5f, d2=%.5f, g=%.5f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # evaluate the model performance, sometimes

    if (i+1) % 5 == 0:
      summarize_performance(i, g_model, d_model, gan_model, dataset, latent_dim)
      df_hist = pd.DataFrame(list_hist, columns=["epoch","batch_per_epo","d_acc_real",
                                                "d_loss_real","d_acc_fake","d_loss_fake","g_loss"])
      plot_history(df_hist)
      df_hist.to_csv(csv_path, mode="w", index=False)



In [11]:
def train_model(latent_dim=500, initial_epoch=0):
  csv_path = "loss.csv"
  if not os.path.exists(csv_path): 
    with open(csv_path,'a+') as f:
       f.write('epoch,batch_per_epo,d_acc_real,d_loss_real,d_acc_fake,d_loss_fake,g_loss')
  os.makedirs('figure_plots', exist_ok=True)
  os.makedirs('model_checkpoints', exist_ok=True)
  os.makedirs('model_summaries', exist_ok=True)

  #get last epoch from  generator last checkpoint
  generator_models = list(filter(lambda x: x,[re.findall('[0-9]+',s.split('.')[0]) for s in os.listdir('model_checkpoints')]))
  last_epoch = max([int(g[0]) for g in generator_models]) if len(generator_models) != 0 else 0

  # size of the latent space is 100 by default
  if last_epoch == 0:
    print("Start training ... ")
    # create the generator
    g_model = define_generator(latent_dim)
    # create the discriminator
    d_model = define_discriminator()
    # create the gan
    gan_model = define_gan(g_model, d_model)
    
  else:
    trim_csv(csv_path, last_epoch)
    
    model_paths = ["discriminator_model.h5",
              "GAN_model_weights.h5","GAN_optimizer_weights.npy" ]
    model_paths = [os.path.join("model_checkpoints", p)  for p in model_paths]
    
    g_model = load_model(os.path.join("model_checkpoints","generator_model_%03d.h5" % (last_epoch)))
    d_model = load_model(model_paths[0])

    gan_model = define_gan(g_model,d_model)
    gan_model.load_weights(model_paths[1])
    gan_model._make_train_function()

    weight_optimizer = np.load(model_paths[2], 
                            allow_pickle=True).tolist()

    gan_model.optimizer.set_weights(weight_optimizer)
    
    initial_epoch = last_epoch
    print("Last trained epoch %03d" % (last_epoch))
    print("Resume training ...")

  # load image data
  dataset = load_real_samples()

  models = [g_model,d_model,gan_model]
  filenames = ['Generator_model.png','Discriminator_model.png','GAN_model.png']
  for (model,fn) in zip(models,filenames):
    plot_model(model, to_file=fn, show_shapes=True, show_layer_names=True)
    save_model_summary(model, os.path.join('model_summaries', fn))
  
  start = time.time()
  train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1200,
        csv_path=csv_path,initial_epoch=initial_epoch)
  print(f"Elapsed time: {time.time() - start} seconds") 

In [13]:
train_model() 

csv trimed at epoch > 5


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Last trained epoch 005
Resume training ...


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>6, 1/112, d1=0.36843, d2=0.17733, g=6.09031
>6, 2/112, d1=0.27548, d2=0.06446, g=5.00614
>6, 3/112, d1=0.28798, d2=0.35351, g=6.18226
>6, 4/112, d1=0.43021, d2=0.02159, g=4.94041
>6, 5/112, d1=0.05667, d2=0.18523, g=5.59404
>6, 6/112, d1=0.24984, d2=0.05577, g=5.48149
>6, 7/112, d1=0.43790, d2=0.11883, g=5.36758
>6, 8/112, d1=0.33803, d2=0.02374, g=4.94906
>6, 9/112, d1=0.22053, d2=0.13733, g=4.97697
>6, 10/112, d1=0.32127, d2=0.11738, g=6.00347
>6, 11/112, d1=0.38045, d2=0.01714, g=5.13684
>6, 12/112, d1=0.19752, d2=0.24629, g=6.89917
>6, 13/112, d1=0.42325, d2=0.01388, g=6.56537
>6, 14/112, d1=0.22664, d2=0.09468, g=6.04193
>6, 15/112, d1=0.25078, d2=0.09149, g=6.30551
>6, 16/112, d1=0.36785, d2=0.10673, g=5.88816
>6, 17/112, d1=0.40933, d2=0.21503, g=6.80632
>6, 18/112, d1=0.42881, d2=0.10799, g=6.70087
>6, 19/112, d1=0.37324, d2=0.03830, g=5.51451
>6, 20/112, d1=0.30076, d2=0.74628, g=9.72988
>6, 21/112, d1=0.77641, d2=0.00328, g=9.49525
>6, 22/112, d1=0.53298, d2=0.04498, g=7.302

KeyboardInterrupt: ignored

In [ ]:
import re
import os
from utils.get_last_epoch import get_last_epoch

ckpt_paths = ["generator_model_%03d.h5","discriminator_model_%03d.h5",
              "GAN_model_weights_%03d.h5","GAN_optimizer_weights_%03d.npy" ]
last_epoch = get_last_epoch('model_checkpoints')
ckpt_paths = [os.path.join("model_checkpoints", p % (last_epoch))  for p in ckpt_paths]
ckpt_paths

In [ ]:
# import pandas as pd
# csv = pd.read_csv("accuracy_loss.csv")
# csv.shape,csv.iloc[-112:,:]

In [ ]:
gan_model.load_weights("/content/drive/My Drive/DCGAN_Batik_Dataset/test/gan_model_weights.h5")
gan_model._make_train_function()

with open('optimizer.pkl', 'rb') as f:
  weight_values = pickle.load(f)

gan_model.optimizer.set_weights(weight_values)
# gan_optimizer_weights = np.load("/content/drive/My Drive/DCGAN_Batik_Dataset/test/gan_optimizer_weights/gan_optimizer_weights.npy",allow_pickle=True)
# gan_model.optimizer.set_weights(gan_optimizer_weights.tolist())
# len(gan_model.optimizer.weights),len(gan_optimizer_weights.tolist()), len(weight_values)


In [ ]:
# l = gan_model.optimizer.get_weights()
# arr = np.asarray(l)
# np.save('optimizer_weights.npy', arr)
# arr == l


In [ ]:
# loaded_arr = np.load('optimizer_weights.npy',allow_pickle=True)
# loaded_arr = loaded_arr.tolist()
# for i in range(len(loaded_arr)):
#   is_equal = np.array_equal(arr[0],l[0])
#   if not is_equal:
#     print(i)
# gan_model.optimizer.set_weights(loaded_arr)
# gan_model.optimizer.get_weights()

In [ ]:
# ckpt_path = "model_checkpoints"
# discriminator_ckpt = list(map(lambda x: x.group(),filter(lambda x : x,[re.match("discriminator_model_[0-9]+.h5",m) for m in os.listdir(ckpt_path)])))
# discriminator_ckpt = list(filter(lambda x: x != "discriminator_model_400.h5" ,discriminator_ckpt))
# GAN_model_weights_ckpt = list(map(lambda x: x.group(),filter(lambda x : x,[re.match("GAN_model_weights_[0-9]+.h5",m) for m in os.listdir(ckpt_path)])))
# GAN_model_weights_ckpt = list(filter(lambda x: x != "GAN_model_weights_400.h5" ,GAN_model_weights_ckpt))
# GAN_optimizer_weights_ckpt = list(map(lambda x: x.group(),filter(lambda x : x,[re.match("GAN_optimizer_weights_[0-9]+.npy",m) for m in os.listdir(ckpt_path)])))
# GAN_optimizer_weights_ckpt = list(filter(lambda x: x != "GAN_optimizer_weights_400.npy" ,GAN_optimizer_weights_ckpt))

# for fn in zip(discriminator_ckpt, GAN_model_weights_ckpt, GAN_optimizer_weights_ckpt):
#   print(fn)
  # os.remove(os.path.join(ckpt_path,fn[0]))
  # os.remove(os.path.join(ckpt_path,fn[1]))
  # os.remove(os.path.join(ckpt_path,fn[2]))


In [ ]:
# generator_models = filter(lambda x: x,[re.findall('[0-9]+',s.split('.')[0]) for s in os.listdir('model_checkpoints')])
# last_epoch = max([int(g[0]) for g in generator_models])
# model_paths = ["discriminator_model.h5",
#               "GAN_model_weights.h5","GAN_optimizer_weights.npy" ]
# model_paths = [os.path.join("model_checkpoints", p)  for p in model_paths]
# g_model = load_model(os.path.join("model_checkpoints","generator_model_%03d.h5" % (last_epoch)))
# d_model = load_model(model_paths[0])

# gan_model = define_gan(g_model,d_model)
# gan_model.load_weights(model_paths[1])
# gan_model._make_train_function()

# weight_optimizer = np.load(model_paths[2], 
#                         allow_pickle=True).tolist()

# gan_model.optimizer.set_weights(weight_optimizer)